In [1]:
import numpy as np
from scipy import optimize
import sympy as sp
from jade import JADifferentialEvolution
from pprint import pprint


```py
s = sp.symbols("s")
t = sp.symbols("t", positive=True)
N = 50
Sn = N**2 / 2


def pade_exp(n, s):
    return (1 + 0.5 * n * s) / (1 - 0.5 * n * s)


G = sp.Array(
    [
        [
            (12.8 * pade_exp(-1, s)) / (16.7 * s + 1),
            (-18.9 * pade_exp(-3, s)) / (21 * s + 1),
        ],
        [
            (6.6 * pade_exp(-7, s)) / (10.9 * s + 1),
            (-19.4 * pade_exp(-3, s)) / (14.4 * s + 1),
        ],
    ]
)

# Fazer em laplace
def wood_berry(u, s):
    g = G
    y = g @ u
    return y


def funcao_objetivo(erro):
    f = sum(k * (np.abs(e1) + np.abs(e2)) for k, (e1, e2) in enumerate(erro)) / Sn
    return f


# Fazer em laplace
def controlador(c, s, erro):
    # Função qualquer aqui só para rodar
    # Depois trocar pelo modelo correto
    u1 = (c[0] + c[1] / s + c[2] * s) * erro[0]
    u2 = (c[3] + c[4] / s + c[5] * s) * erro[1]
    return np.array([u1, u2]).T


def inverse_laplace(sis, s, t):
    sis = sp.N(sp.polys.partfrac.apart(sis, s, full=True).doit(), 5)
    for sisf in sis.args:
        v1 = 0
        v2 = 0
        if len(sisf.args):
            v1 = sisf.args[0]
            sisf2 = 1 / sisf.args[1]
            if len(sisf2.args):
                v2 = sisf2.args[0]
        yield v1 * sp.exp(-v2 * t)


# Transformar em função de tempo
def sistema(c, s, t, erro):
    u = controlador(c, s, erro)  # Tenho que mudar aqui
    y = wood_berry(u, s)  # Tenho que mudar aqui
    return [sum(inverse_laplace(y, s, t)) for y in y]


def evaluate_sistem(c, s, t, N):
    erro = np.array([1, 1])
    sis = sistema(c, s, t, erro)
    erros = []
    for k in range(N):
        val0 = sis[0] if isinstance(sis[0], int) else sis[0].evalf(subs={t: k})
        val1 = sis[1] if isinstance(sis[1], int) else sis[1].evalf(subs={t: k})
        erro = np.array(
            [
                float(val0) - 0.96,
                float(val1) - 0.05,
            ]
        )
        sis = sistema(c, s, t, erro)
        erros.append(erro)
    itae = funcao_objetivo(erros)
    print(c)
    print(itae)
    # input()
    print()
    return itae


bounds = ((-0.2, 0.2),) * 6
c = np.array([0.19, 0, 0, -0.075, 0, 0])
```


In [ ]:
res = optimize.minimize(
    evaluate_sistem, x0=c, args=(s, t, N), bounds=bounds, method="L-BFGS-B"
)
# evaluate_sistem(c, s, t, N)


[ 0.19   0.     0.    -0.075  0.     0.   ]
9.268941125856513e+17

[ 0.19000001  0.          0.         -0.075       0.          0.        ]
4962431223593709.0

[ 1.9e-01  1.0e-08  0.0e+00 -7.5e-02  0.0e+00  0.0e+00]
1.5680352206881588e+46

[ 1.9e-01  0.0e+00  1.0e-08 -7.5e-02  0.0e+00  0.0e+00]
4.3445127440856033e+18

[ 0.19        0.          0.         -0.07499999  0.          0.        ]
1.5480459420521838e+18

[ 1.9e-01  0.0e+00  0.0e+00 -7.5e-02  1.0e-08  0.0e+00]
1.44788099311267e+52

[ 1.9e-01  0.0e+00  0.0e+00 -7.5e-02  0.0e+00  1.0e-08]
7.743251010078979e+19

[ 1.90000000e-01 -2.00000000e-01 -4.35910952e-29 -7.50000000e-02
 -2.00000000e-01 -9.75815020e-28]
1.2194050501221017e+83

[ 1.90000010e-01 -2.00000000e-01 -4.35910952e-29 -7.50000000e-02
 -2.00000000e-01 -9.75815020e-28]
5.8607453101856716e+82

[ 1.90000000e-01 -1.99999990e-01 -4.35910952e-29 -7.50000000e-02
 -2.00000000e-01 -9.75815020e-28]
5.963400332004554e+78

[ 1.9000000e-01 -2.0000000e-01  1.0000000e-08 -7.5000000

In [3]:
class WoodBerry:
    def __init__(self):
        self.s = sp.symbols("s")
        self.t = sp.symbols("t", positive=True)
        self.N = 50
        self.Sn = self.N**2 / 2

        self.bounds = np.array(
            [
                [-0.2, 0.2],
            ]
            * 4
        )

        self.active_constrains = False

        self.G = sp.Array(
            [
                [
                    (12.8 * self.pade_exp(-1)) / (16.7 * self.s + 1),
                    (-18.9 * self.pade_exp(-3)) / (21 * self.s + 1),
                ],
                [
                    (6.6 * self.pade_exp(-7)) / (10.9 * self.s + 1),
                    (-19.4 * self.pade_exp(-3)) / (14.4 * self.s + 1),
                ],
            ]
        )

    def pade_exp(self, n):
        return (1 + 0.5 * n * self.s) / (1 - 0.5 * n * self.s)

    def wood_berry(self, u):
        y = self.G @ u
        return y

    def funcao_objetivo(self, erro):
        f = (
            sum(k * (np.abs(e1) + np.abs(e2)) for k, (e1, e2) in enumerate(erro))
            / self.Sn
        )

        return f

    def inverse_laplace(self, sis):
        sis = sp.N(sp.polys.partfrac.apart(sis, self.s, full=True).doit(), 5)
        for sisf in sis.args:
            v1 = 0
            v2 = 0
            if len(sisf.args):
                v1 = sisf.args[0]
                sisf2 = 1 / sisf.args[1]
                if len(sisf2.args):
                    v2 = sisf2.args[0]
            yield v1 * sp.exp(-v2 * self.t)

    # Fazer em laplace
    def controlador(self, c, erro):
        # Função qualquer aqui só para rodar
        # Depois trocar pelo modelo correto
        # u1 = (c[0] + c[1] / self.s + c[2] * self.s) * erro[0]
        # u2 = (c[3] + c[4] / self.s + c[5] * self.s) * erro[1]
        u1 = (c[0] + c[1] * self.s) * erro[0]
        u2 = (c[2] + c[3] * self.s) * erro[1]
        return np.array([u1, u2]).T

    # Transformar em função de tempo
    def sistema(self, c, erro):
        u = self.controlador(c, erro)  # Tenho que mudar aqui
        y = self.wood_berry(u)  # Tenho que mudar aqui
        return [sum(self.inverse_laplace(y0)) for y0 in y]

    def evaluate_sistem(self, x):
        erro = np.array([1, 1])
        sis = self.sistema(x, erro)
        erros = []
        for k in range(self.N):
            val0 = sis[0] if isinstance(sis[0], int) else sis[0].evalf(subs={self.t: k})
            val1 = sis[1] if isinstance(sis[1], int) else sis[1].evalf(subs={self.t: k})
            erro = np.array(
                [
                    float(val0) - 0.96,
                    float(val1) - 0.05,
                ]
            )
            sis = self.sistema(x, erro)
            erros.append(erro)
        itae = self.funcao_objetivo(erros)
        print(x)
        print(itae)
        # input()
        print()
        return itae

    def problem(self, x):
        return self.evaluate_sistem(x)


In [4]:
wb = WoodBerry()


In [12]:
with open("./trabalho_05_jade.txt") as f:
    results = f.read()
results = list(set(results.split("\n\n")))
results = [res.splitlines() for res in results]
results = [
    (
        list(
            map(
                float,
                list(
                    pesos.replace("[", "")
                    .replace("]", "")
                    .strip()
                    .replace("   ", " ")
                    .replace("  ", " ")
                    .replace("  ", " ")
                    .replace("  ", " ")
                    .replace("  ", " ")
                    .split(" ")
                ),
            )
        ),
        float(valor),
    )
    for pesos, valor in results
    if float(valor) < 1000
]


In [13]:
X0 = np.concatenate(
    [
        np.array([pesos for pesos, valor in results]),
        np.random.uniform(low=-0.2, high=0.2, size=(60-len(results), 4)),
    ]
)

In [14]:
N = 30
G = 200
config = {"case1": {"mutation": 0.6, "recombination": 0.8}}


def run(algorithm, problem, N, G, config):
    res = dict()
    for case in config:
        res[case] = []
        for i in range(N):
            alg = algorithm(
                problem.problem, problem.bounds, seed=i, G=G, **config[case], X=X0
            )
            alg.run()
            res[case].append(alg.fun)
    return res


res_jade = run(algorithm=JADifferentialEvolution, problem=wb, N=N, G=G, config=config)
print("Finalizado")


[-0.07911393 -0.09650316  0.00055288 -0.00057678]
118.21519310215297

[ 1.66228716e-01 -1.37879716e-01  1.46174968e-04 -3.29763673e-04]
2786.3461454467233

[0.0724852  0.13714123 0.00040602 0.00044692]
58.44011618494175

[ 2.67525705e-02  1.28414204e-01 -5.18219909e-04 -1.16250920e-04]
279.13710546732386

[-0.00764424  0.08377265  0.00042519  0.00021014]
3211.51116255785

[-0.07566155  0.03522902 -0.0005578  -0.0005362 ]
58023.13483090863

[-7.64423883e-03 -2.00000000e-01  9.93905246e-04 -1.10861624e-04]
253888.04139332552

[ 0.00515627 -0.19039385  0.00038466 -0.00042523]
2328.3143063161965

[-0.07911393 -0.02197298  0.00024533 -0.00012652]
9.896637048459537

[-0.07784398 -0.11702903 -0.00095271  0.00019897]
46108.86914191424

[-2.00000000e-01 -9.41821691e-02  9.94436247e-04  1.87119780e-04]
48897.831899604316

[ 0.07057434  0.07705238 -0.00016635 -0.00025304]
209.6004061237518

[-0.1498572   0.10657763  0.00172854  0.00119915]
312776.88932358916

[ 0.02455808  0.01635199  0.00094    